<h1>Interacting with the VAMPS-API</h1>
<h2>This notebook:</h2>
<ul>
<li><strong>Logs into VAMPS account </strong> if you don't have an account, use guest account (Username: 'guest', Password: 'guest')</li>
<li><strong>Uses a config file</strong></li>
Either specify an existing config file to use <strong>OR</strong> edit 'config' in the code below to match your preferences
<li><strong>Gets dataset ID's for specified dataset</strong></li>
<li><strong>Displays specified visualization</strong></li>
<li><strong>Saves visualization locally to your computer</strong></li>
<li><strong>Can upload config to VAMPS for later use</strong></li>
</ul>
<h2>To use this notebook:</h2>
<ul><strong><li>Press run cell button</li>
<li>Enter information if promtped</li>
<li>Run next cell</li></strong></ul>

In [12]:
"""
Panel-1 
Define config variable; Select project; Select Image;
Author: AAV / SR
vamps2.mbl.edu
"""

import os,sys
import requests
from bs4 import BeautifulSoup   # parser for html
import json, string, getpass

#to allow both Python 2 and 3
try:
    input = raw_input  
except NameError: #Python 3
    pass

#get VAMPS username and password
user = input("Enter your VAMPS username: ")
pw = getpass.getpass("Enter your VAMPS password: ")

conn = {'user': user,
        'passwd': pw,
         # vamps:             https://vamps2.mbl.edu
         # vampsdev (private) http://vampsdev.jbpc-np.mbl.edu:8124 
         # localhost:         http://localhost:3000 
        'hosturl':'https://vamps2.mbl.edu'
       }

#attempt login to VAMPS
s = requests.Session()
r = s.post(conn['hosturl']+'/users/login', data={'username':conn['user'], 'password':conn['passwd']})

#exit program if login unsuccessful
if r.url == 'https://vamps2.mbl.edu/users/login':
    sys.exit('Login not successful')
elif r.url == 'https://vamps2.mbl.edu/':
    print('Login successful')

upload = input("Do you want to use an already existing config file? ('Y' or 'N'): ")

#to upload config: 
if upload[0].capitalize() == "Y":
    file = input('Enter JSON Config File: ')
    with open(file) as f:        
        config = json.load(f)
else:
    print("Edit 'config' below to match preferences before running cell")


Enter your VAMPS username: ruzics
Enter your VAMPS password: ········
Login successful
Do you want to use an already existing config file? ('Y' or 'N'): N
Edit 'config' below to match preferences before running cell


In [13]:
"""
Panel-2 
Set config file if one wasnt uploaded
Get dataset ID's 
"""

if upload == "N":
    #edit config, comments next to values show allowed different options
    config = {
        "api":"1",
        "source":"VAMPS-API",
        "update_data":1,
        "normalization":"none",              # none, maximum, frequency
        "selected_distance":"jaccard",  # morisita-horn, jaccard, kulczynski, canberra bray-curtis
        "tax_depth":"klass",                 # domain, phylum, klass, order, family, genus, species, strain
        "domains":["Archaea","Bacteria","Eukarya","Organelle","Unknown"],   #["Archaea","Bacteria","Eukarya","Organelle","Unknown"],
        "include_nas":"yes",                  # yes or no
        "min_range":0,                        # integer 0-99
        "max_range":100,                      # integer 1-100

          # Must be a valid project - with correct permissions for the above user.
        'project':'DCO_MAC_Bv6',   

          # Currently avalable: "dheatmap", "piecharts", "barcharts", "counts_matrix", "metadata_csv", "adiversity", "fheatmap", "dendrogram" 
        'image':'fheatmap'
    }
    
# get project ids:
r = s.post(conn['hosturl']+'/api/get_dids_from_project', timeout=15, data=config)  
config['ds_order'] = r.text
print(config['ds_order'])

[239132,239128,239129,239130,239131,239133,239134,239135]


In [14]:
"""
Panel-3
Creates remote configuration and gets timestamp (file_prefix) 
"""
# Get timestamp (filename prefix):
r = s.post(conn['hosturl']+'/visuals/view_selection', timeout=15, data=config)
soup = BeautifulSoup(r.text, "lxml")  # html5lib  lxml html.parser

ts = soup.find(id="ts_for_bs").string

In [15]:
"""
Panel-4
Saves matrx_file which is integral to vamps images
Saves image file
"""

import json
biom_matrix_file = ts+'_count_matrix.biom'
url = conn['hosturl']+"/"+biom_matrix_file
response = requests.get(url, stream=True)
response.raise_for_status()
out_file = biom_matrix_file
with open(out_file, "wb") as handle:
    for block in response.iter_content(1024):
        handle.write(block)


r = s.post(conn['hosturl']+'/api/create_image', timeout=30, data=config)

try:
    result = json.loads(r.text)
except:
    print(r.text)
    sys.exit()
local_filename = result['filename']
return_result = result['html']
print(local_filename)
remote_file_name = conn['hosturl']+"/"+local_filename

r = requests.get(remote_file_name, stream=True)
with open(local_filename, 'wb') as f:
    f.write(r.content)
print('Done writing local file:',local_filename)

ruzics_1500568197793_fheatmap.svg
Done writing local file: ruzics_1500568197793_fheatmap.svg


In [18]:
"""
Panel-5
Show image
Choice to upload config file back to VAMPS 
"""

upload_to_vamps = input("Do you want to upload config file to VAMPS for later use?('Y' or 'N'): ")
if upload_to_vamps[0].capitalize() == 'Y':
    dataset_ids = config['ds_order'].strip('[')
    dataset_ids = dataset_ids.strip(']')
    dataset_ids = dataset_ids.split(',')
    vamps_config = {
      "source":"VAMPS",
      "image":config['image'],
      "post_items":
      { 
        "normalization":config['normalization'],
        "selected_distance":config['selected_distance'],
        "tax_depth":config['tax_depth'],
        "domains":config['domains'],
        "include_nas":config['include_nas'],
        "min_range":config['min_range'],
        "max_range":config['max_range']
      },
        "id_name_hash":
        {
          "ids":dataset_ids
        }
    }
    upload = {
    'upload_files': json.dumps(vamps_config)
    }
    data = {
    'from_upload_configuration_file' : '1'
    }
    r = s.post('https://vamps2.mbl.edu/visuals/view_selection', data=data,files=upload)

ready_images = ["dheatmap", "piecharts", "barcharts", "counts_matrix", "metadata_csv", "adiversity", "fheatmap", "dendrogram" ]

from IPython.core.display import display, HTML
out = ''
if config['image'] in ready_images:
    if config['image'] == 'metadata_csv':
        print(return_result)
    else:
        out = HTML("<style>.container { width:100% !important; }</style>"+return_result)
else:
    out = 'not implemented yet'
out

Do you want to upload config file to VAMPS for later use?('Y' or 'N'): Y
